# BikeSharing Data Model
### Data Engineering Capstone Project

In [2]:
# Do all imports and installs here
import pandas as pd

In [3]:
# Read in the data here
bay_area = pd.read_csv("C:\\Users\\Hitesh Somani\\Documents\\udacity\\capstone_project\\baywheels_bay_area\\201908-bay-area-tripdata.csv")
nyc = pd.read_csv("C:\\Users\\Hitesh Somani\\Documents\\udacity\\capstone_project\\citibike_nyc\\201908-nyc-tripdata.csv")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

- We have few missing in both the tables 
- The column names are not same
- There is no duplication
- mapping of gender column

In [4]:
# Performing cleaning tasks here
bay_area.isnull().sum()/ bay_area.shape[0]

duration_sec               0.000000
start_time                 0.000000
end_time                   0.000000
start_station_id           0.000000
start_station_name         0.000000
start_station_latitude     0.000000
start_station_longitude    0.000000
end_station_id             0.000000
end_station_name           0.000000
end_station_latitude       0.000000
end_station_longitude      0.000000
bike_id                    0.000000
user_type                  0.000000
member_birth_year          0.188965
member_gender              0.188951
bike_share_for_all_trip    0.000000
dtype: float64

In [5]:
bay_area.dtypes

duration_sec                 int64
start_time                  object
end_time                    object
start_station_id             int64
start_station_name          object
start_station_latitude     float64
start_station_longitude    float64
end_station_id               int64
end_station_name            object
end_station_latitude       float64
end_station_longitude      float64
bike_id                      int64
user_type                   object
member_birth_year          float64
member_gender               object
bike_share_for_all_trip     object
dtype: object

In [6]:
nyc.isnull().sum()/nyc.shape[0]

tripduration               0.000000
starttime                  0.000000
stoptime                   0.000000
start station id           0.000038
start station name         0.000038
start station latitude     0.000000
start station longitude    0.000000
end station id             0.000038
end station name           0.000038
end station latitude       0.000000
end station longitude      0.000000
bikeid                     0.000000
usertype                   0.000000
birth year                 0.000000
gender                     0.000000
dtype: float64

We have few missing in both the tables and also the column names are not same

In [7]:
# Check duplication. So no duplication exists
print(bay_area.duplicated().sum())
print(nyc.duplicated().sum())

0
0


In [10]:
bay_area['member_gender'].unique()

array([nan, 'Female', 'Male', 'Other'], dtype=object)

In [11]:
nyc['gender'].unique()

array([2, 1, 0], dtype=int64)